In [ ]:
from DataLoader.dataloader  import upload_trading_idea


# Convert to timframe format from string to int in minute
def process_timeframe(row):
    for k, s in zip(["D", "W", "M"], [24, 5*24, 5*24*30]):
        if k in row:
            row = int(row.split(k)[0])*s*60
            break
    
    return int(row)*60


data    = pd.read_csv("Put Path to Data")   # data pandas.DataFrame

# Rename the columns to fit the database columns name
data    = data.rename(columns={"Pdate": "date", "Acc_Type": "account_type", "FinancialMarkets": "market", "Symbol": "code"})
data    = data.rename(columns={k: k.lower() for k in data.columns})                         # Set all the columns name to lower case

# Convert to date string to pandas datetime
data["date"]        = pd.to_datetime(data["date"], errors='coerce')
data                = data.loc[data["date"].dropna().index]                                 # Get rid of nan due to bad convertion of date to datetime
data["timestamp"]   = data.apply(lambda x: int(x["date"].timestamp()), axis=1)              # Gather unix timestamp for all date
data['date']        = data['date'].dt.tz_localize('UTC').dt.strftime("%Y-%m-%d %H:%M:%S")   # Set datetime to GMT 0 / UCT time zone
data["timeframe"]   = data.apply(lambda x: process_timeframe(x["timeframe"]), axis=1)       # Convert timeframe columns to second

# Upload the pandas dataframe to the database
upload_trading_idea(data)
